In [1]:
pip install codecarbon

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
! codecarbon init


Welcome to CodeCarbon, here is your experiment id:
73c54d65-6fc8-4a34-bef1-c56b3a73de67

CodeCarbon automatically added this id to your local config: ./.codecarbon.config



In [ ]:
from codecarbon import track_emissions
@track_emissions()
def your_function_to_track():
  # your code